# Automating Video Content Metadata with Frame Analysis

## Setting Up a New Notebook for Content Analysis

To extend our pipeline, we will create a new Jupyter Notebook dedicated to generating video descriptions and keywords automatically. This notebook will build on the metadata files produced by the previous steps (e.g., batch_metadata.json, video_inventory.csv). By separating concerns, the first notebook handles file discovery and metadata extraction, while this new notebook focuses on content analysis using video frames.

**Key idea:** We will use video frame analysis with AI models to generate a descriptive caption and a set of keywords for each video. The process will be automated, ensuring each clip gets a unique description (15-200 characters, at least 5 words) and 8-49 relevant keywords (comma-separated, no duplicates) as required by BlackBox Global.

## Loading Existing Metadata

Begin by loading the metadata produced by the previous notebook. This gives us the list of videos (in a pandas DataFrame or JSON) that need descriptions and keywords:

In [ ]:
# Cell 1: Load existing metadata and dataframes
import json
import pandas as pd

# Load the batch metadata JSON and video inventory CSV from the previous steps
metadata_path = "batch_metadata.json"   # or the path where it was saved
inventory_path = "video_inventory.csv"

with open(metadata_path, 'r') as f:
    batch_metadata = json.load(f)
df_videos = pd.read_csv(inventory_path)

print(f"Loaded {len(df_videos)} video records from inventory.")

This will load df_videos containing columns like batch_name, filename, path, full_path, etc. We will enhance this DataFrame with new columns for Description and Keywords.

# Extracting Representative Video Frames

To describe a video, we need to analyze its visual content. A common approach is to extract one or more representative frames from the video:
1. Choose a frame time – For each video, decide on a timestamp to capture a frame. For example, the midpoint of the video or a few seconds in (assuming the video isn’t just black at start). This can be done with OpenCV or FFmpeg in Python.
2. Extract the frame – Use OpenCV (cv2.VideoCapture) to read the video file and retrieve a frame at the chosen time. For example, to get the middle frame:

In [ ]:
import cv2

def get_video_frame(path, time_sec=1.0):
    cap = cv2.VideoCapture(path)
    cap.set(cv2.CAP_PROP_POS_MSEC, time_sec * 1000)  # jump to time_sec (in milliseconds)
    success, frame = cap.read()
    cap.release()
    if success:
        return frame  # frame is a NumPy array (H x W x 3)
    else:
        return None

We set CV_CAP_PROP_POS_MSEC to the desired timestamp (in milliseconds) and read a frame. We may adjust time_sec if needed (e.g., 0.5 * duration for mid-video). For reliability, you might also capture multiple frames (start, middle, end) and choose the clearest or use all for analysis.

3. Store or process the frame – We won’t display the frame in the notebook, but we will feed it into an AI model for captioning. If needed, we can use PIL to convert the frame array to an image object:

In [ ]:
from PIL import Image

frame = get_video_frame(video_path, time_sec=video_midpoint)
if frame is not None:
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

(Make sure to handle cases where frame extraction might fail, e.g., if a video file is short or unreadable.)

## Generating Descriptions with an AI Model

With a representative frame in hand, we can use an image captioning model to generate a natural-language description of the scene:
- Model choice: A state-of-the-art open-source model for image captioning is BLIP (Bootstrapping Language-Image Pre-training) by Salesforce. It can produce descriptive captions for images. Alternatively, other models like OpenAI’s CLIP with a captioning head or image-to-text pipelines on HuggingFace can be used. For our pipeline, we’ll use Hugging Face’s Transformers library with a pre-trained image captioner.
- Setup captioning pipeline: We can use the Transformers pipeline for image-to-text or manually load the BLIP model. For example:

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration

# Load the BLIP model and processor (this may download weights if not cached)
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

def generate_caption(image: Image.Image) -> str:
    inputs = processor(image, return_tensors="pt")
    output_ids = model.generate(**inputs)
    caption = processor.decode(output_ids[0], skip_special_tokens=True)
    return caption

Alternatively, using the pipeline API:

In [ ]:
from transformers import pipeline
captioner = pipeline("image-to-text", model="Salesforce/blip-image-captioning-base")
caption = captioner(img)  # img is a PIL Image or file path
description_text = caption[0]['generated_text']

- Generate description: Once the model is loaded, iterate over each video, extract a frame, and get a caption:

In [ ]:
descriptions = []
for idx, row in df_videos.iterrows():
    frame = get_video_frame(row['full_path'], time_sec=1.0)
    if frame is None:
        descriptions.append("")  # if extraction failed, leave blank or handle accordingly
        continue
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    caption = generate_caption(img)
    descriptions.append(caption)
    print(f"Caption for {row['filename']}: {caption}")

This will produce a descriptive sentence for each frame, for example:
- “A rustic well pump dripping water in a rural yard.”
- “A man using a hand pump to wash his hands with water.”
Ensure that the generated captions meet BlackBox’s requirements (at least 5 words and >=15 characters). If a caption is too short, you might prompt the model with a prefix (e.g., “A photo of” …) to encourage detail, or post-process by adding context.

# Generating Keywords from Descriptions

With a description available, we can derive a set of keywords. The goal is 8 to 49 relevant keywords, separated by commas, without repetition. Here’s how to approach it:
1. Extract key terms from the caption: Take the generated caption and split it into words. Remove common stopwords and very short words. For example, from “A rustic well pump dripping water in a rural yard” we might extract words like [“rustic”, “well”, “pump”, “dripping”, “water”, “rural”, “yard”]. This is a starting point.
2. Augment with domain-specific terms: The caption might not explicitly list all relevant concepts. We can add:
- Synonyms or related terms (e.g., “well pump” might also imply “hand pump”, “water pump”, “outdoor”, “metal”, etc.).
- Environment or category terms (if known, e.g., “Infrastructure”, “Agriculture”, “Outdoors”).
- If the video has a person, include “man”, “woman”, “people” as appropriate.
3. Automate keyword suggestions: There are a few ways to do this:
- Use a pre-trained image tagging model or object detection (like YOLOv5 or Detectron) on the frame to get object labels. For instance, an object detector could identify “person”, “water”, “pump” which you ensure are in the keywords.
- Use an NLP approach: take the caption and use a library (spaCy or NLTK) to find noun phrases or named entities.
- For a more advanced route, use a language model (GPT-style) to suggest a list of keywords given the caption (this could be overkill but possible if integrated via an API).
For simplicity, let’s derive keywords by processing the caption text:

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

STOPWORDS = set(stopwords.words('english'))
def caption_to_keywords(caption: str, extra_terms: list[str] = None) -> list[str]:
    words = [w.strip(",.").lower() for w in caption.split()]
    # Filter out short words and stopwords
    candidates = [w for w in words if len(w) > 2 and w not in STOPWORDS]
    # Add any extra terms (like category or known context)
    if extra_terms:
        candidates += [term.lower() for term in extra_terms]
    # Deduplicate while preserving order
    seen = set()
    keywords = []
    for w in candidates:
        if w not in seen:
            keywords.append(w)
            seen.add(w)
    return keywords

We can call caption_to_keywords() for each caption. We might pass extra_terms like the batch name or category if relevant. For example, for the “well pump” videos, we could add extra_terms=["outdoors", "infrastructure"] to ensure those appear.

4. Ensure minimum count: If the resulting keyword list has fewer than 8 items, consider adding some generic related words or splitting phrases. For instance, if we have “well pump” as one phrase, we might split into “well, pump” as separate keywords to increase count (unless repetition rules forbid splitting a compound concept—BlackBox typically expects distinct concepts, so splitting is okay if they are distinct words).
5. Format as required: Join the keywords into a comma-separated string:

In [ ]:
keywords_list = []
for caption in descriptions:
    terms = caption_to_keywords(caption)
    # Ensure at least 8 keywords (add filler if needed, or handle in caption_to_keywords)
    if len(terms) < 8:
        terms = terms + ["stock", "footage"]  # (placeholder or contextual filler to reach count)
        terms = terms[:8]
    keywords_str = ", ".join(terms[:49])  # ensure not exceeding 49
    keywords_list.append(keywords_str)

For our examples:
- Description: “A rustic well pump dripping water in a rural yard.” → Keywords: "rustic, well, pump, dripping, water, rural, yard, outdoor"
- Description: “A man using a hand pump to wash his hands with water.” → Keywords: "man, hand, pump, wash, hands, water, outdoor, person"
We included “outdoor/person” which might come from object detection or context. These keywords align with BlackBox guidelines (all lowercase, no repetition, clearly separated by commas).

Integrating Descriptions and Keywords into the DataFrame

Now that we have lists of descriptions and keywords corresponding to each video, we can add them to our df_videos DataFrame:

In [ ]:
df_videos['Description'] = descriptions
df_videos['Keywords'] = keywords_list

# Quick sanity check
print(df_videos[['filename', 'Description', 'Keywords']].head(5))

This will attach the new metadata to each video entry. At this point, we have an enhanced DataFrame that includes everything: batch name, file path, size, and the newly generated description and keywords.

## Updating the BlackBox Metadata Template

In the original pipeline, we used an Excel template (BlackBox Metadata Template.xlsx) and appended rows with metadata for each video, including placeholders for Description and Keywords. Now we will populate those fields with our generated data:

- Re-open or create a DataFrame for BlackBox rows: We can reuse the generate_blackbox_rows logic, but instead of dummy text, use the actual Description and Keywords from df_videos. For example:

In [ ]:
# Assuming df_videos has at least columns: batch_name, filename, Description, Keywords
bb_rows = []
for _, row in df_videos.iterrows():
    file_path = f"{row['batch_name']}/{row['filename']}".replace("\\", "/")
    bb_rows.append({
        "File Name": file_path,
        "Description (min 15, max 200 characters, must be least 5 words)": row['Description'],
        "Keywords (min 8, max 49, separated by comma, and no repetition)": row['Keywords'],
        "Category (use dropdown menu)": "Infrastructure" if "pump" in row['Keywords'] else "Nature",  # example category logic
        "Editorial (use dropdown menu)": False
        # ... any other required fields, possibly left default or blank for now
    })
df_bb = pd.DataFrame(bb_rows)

In the above snippet, we included a simple category assignment (for example purposes: videos with “pump” were tagged Infrastructure). In practice, you might want to map batch names or certain keywords to appropriate categories manually or via a lookup, since category selection might require human context (e.g., “Infrastructure”, “People”, “Nature”, etc.). Editorial flag we set to False by default unless we know the footage is editorial.

- Write back to Excel: Now append or merge with the BlackBox template Excel. We can open the existing template and replace or append the new rows:

In [ ]:
template_path = "BlackBox Metadata Template MAY 10 2024.xlsx"
df_template = pd.read_excel(template_path, sheet_name="template")

# Remove placeholder rows if any, then append new ones
# (If the template had only headers or example row, ensure not to duplicate)
df_combined = pd.concat([df_template, df_bb], ignore_index=True)
with pd.ExcelWriter(template_path, engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
    df_combined.to_excel(writer, sheet_name="template", index=False)
print("✅ Updated Excel metadata template with generated descriptions and keywords.")

This will fill the Excel sheet with proper descriptions and keywords for each file. We should verify the length and word-count constraints here (e.g., ensure descriptions are not over 200 characters and at least 15, keywords count etc.). Minor adjustments like truncating a too-long description or splitting/joining keywords might be needed to strictly conform to BlackBox rules.

- Export XML if needed: The previous notebook showed how to create metadata.xml from the Excel. We can re-run that export step so the XML includes our new descriptions and keywords:

In [ ]:
# Convert df_combined (or df_template after adding new data) to XML
root = ET.Element("MediaMetaData")
for _, row in df_combined.iterrows():
    clip = ET.SubElement(root, "Clip")
    for col, val in row.items():
        tag = col.replace(" ", "").replace("(", "").replace(")", "").replace(",", "").replace("-", "")
        child = ET.SubElement(clip, tag)
        child.text = str(val)
tree = ET.ElementTree(root)
tree.write("metadata.xml", encoding="utf-8", xml_declaration=True)
print("✅ Exported updated metadata.xml for BlackBox Global")

# Now metadata.xml contains the enriched metadata ready for BlackBox upload.

Summary of the Automated Workflow:

- Notebook 1 (Batch & File Discovery): Discovers video files, organizes them by batch, and saves basic metadata (file names, paths, sizes) to JSON/CSV/Excel.
- Notebook 2 (Content Analysis for Metadata): Loads the above metadata, then for each video:
- Extracts a representative frame.
- Generates a natural-language Description of the scene using an AI captioning model.
- Derives a list of Keywords from the caption (and optional object detection or domain knowledge).
- Populates these into the DataFrame and updates the metadata files (Excel, XML) for BlackBox submission.
- Result: Each stock footage clip now has an automatically generated description and keyword list that can be reviewed or lightly edited, drastically speeding up the metadata authoring process.

## Future Improvements
- Frame selection: For more accuracy, analyze multiple frames per video (beginning, middle, end) and combine insights. This helps if the content changes during the clip.
    
- Caption refinement: Use prompt engineering or a larger model to get more detailed captions if needed. Ensure the description sounds natural and appealing to customers.
    
- Keyword optimization: Integrate a thesaurus or use WordNet to expand keywords. We could also filter out any words not allowed by stock sites (e.g., trademarked terms, etc., if applicable).
    
- Validation: Incorporate checks for BlackBox rules (min/max lengths, no forbidden characters). Possibly integrate the BlackBox Excel’s data validation rules by cross-verifying the output.
    
- Manual review hooks: Even with automation, allow for a manual review step where a human can quickly scan and adjust descriptions/keywords if the AI output isn’t perfect. The framework above ensures the heavy lifting is done, and only fine-tuning might be needed.

By following this approach, we leverage AI to automate the tedious part of metadata creation for stock videos. The result is a scalable pipeline where adding new batches of videos will automatically yield properly formatted metadata (Descriptions and Keywords) that align with BlackBox Global’s requirements, ready for upload.